9. Imagine you are a data analyst working with an e-commerce company that sells automobile products. The company has provided you with an automobile sales dataset named auto sales.csv. You are assigned with the following tasks given below :

a. Write a Spark SQL query to calculate the average price for each product code.
b. Write a Spark SQL query to determine the product line that, on average, has the highest sales price.
c. Write a Spark SQL query to identify the top 3 customers who placed the highest number of orders. Display their customer names and the order count.
d. Write a Spark SQL query to identify orders where the sales amount is high, but the quantity ordered is relatively low.
e. Write a Spark SQL query to find the latest order date for each customer.
NOTE : You have to make use of Azure Databricks in the following question. For submission download the databricks notebook and upload it onto your drive and share with us. Make sure you have given appropriate permissions for evaluation of the file.


In [1]:
import findspark

findspark.init()
findspark.find()


'C:\\Program Files\\spark\\spark-3.4.1-bin-hadoop3'

In [5]:
from pyspark.sql import SparkSession, functions as f

spark = SparkSession.builder.appName("AutoSales").getOrCreate()
spark


In [6]:
df = spark.read.csv("./Auto-Sales-data.csv", inferSchema=True, header=True)
df.show()


+-----------+---------------+---------+---------------+-------+----------+--------------------+-------+-----------+----+-----------+--------------------+----------------+--------------------+-----------+----------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES| ORDERDATE|DAYS_SINCE_LASTORDER| STATUS|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|       CITY|POSTALCODE|  COUNTRY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+----------+--------------------+-------+-----------+----+-----------+--------------------+----------------+--------------------+-----------+----------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0|24-02-2018|                 828|Shipped|Motorcycles|  95|   S10_1678|   Land of Toys Inc.|      2125557818|897 Long Airport ...|        

In [7]:
# a. Write a Spark SQL query to calculate the average price for each product code.
df.groupBy("PRODUCTCODE").agg(f.avg("PRICEEACH").alias("AVG_PRICE")).show()


+-----------+------------------+
|PRODUCTCODE|         AVG_PRICE|
+-----------+------------------+
|   S18_4600|118.08037037037042|
|   S18_1749| 156.3819047619048|
|   S12_3891|159.96653846153848|
|   S18_2248| 68.05714285714286|
|  S700_1138| 71.84384615384614|
|   S32_1268|104.82222222222224|
|   S12_1099|163.72458333333333|
|   S18_2795|156.26960000000003|
|   S24_1937|          56.63125|
|   S32_3522| 79.62192307692308|
|   S18_1097| 109.1453846153846|
|   S18_1662|146.43115384615388|
|   S12_1666|140.92115384615386|
|   S24_3969| 45.33571428571428|
|   S24_1578|104.83083333333332|
|   S24_4048|117.38269230769232|
|   S18_3320|100.19416666666667|
|   S24_3816|           81.0728|
|   S18_3136|100.35959999999999|
|   S32_2509| 63.80464285714286|
+-----------+------------------+
only showing top 20 rows



In [30]:
# b. Write a Spark SQL query to determine the product line that, on average, has the highest sales price.
df.groupBy("PRODUCTLINE").agg(f.avg("sales").alias("average_sales")).sort(
    f.desc("average_sales")
).show(1)


+------------+-----------------+
| PRODUCTLINE|    average_sales|
+------------+-----------------+
|Classic Cars|4049.387291886192|
+------------+-----------------+
only showing top 1 row



In [31]:
# c. Identify the top 3 customers who placed the highest number of orders. Display their customer names and the order count.
df.groupBy("CUSTOMERNAME").agg(f.sum("QUANTITYORDERED").alias("ORDER_COUNT")).sort(
    f.desc("ORDER_COUNT")
).show(3)


+--------------------+-----------+
|        CUSTOMERNAME|ORDER_COUNT|
+--------------------+-----------+
|Euro Shopping Cha...|       9327|
|Mini Gifts Distri...|       6366|
|Australian Collec...|       1926|
+--------------------+-----------+
only showing top 3 rows



In [34]:
# d. Identify orders where the sales amount is high, but the quantity ordered is relatively low.
high_sales_threshold = (
    df.select(f.expr("percentile_approx(sales, 0.75)"))
    .rdd.flatMap(lambda x: x)
    .collect()[0]
)
low_quantity_threshold = (
    df.select(f.avg("quantityordered")).rdd.flatMap(lambda x: x).collect()[0] * 0.2
)
filtered_orders = df.filter(
    (df["sales"] > high_sales_threshold)
    & (df["quantityordered"] < low_quantity_threshold)
)
filtered_orders.show()


+-----------+---------------+---------+---------------+-----+---------+--------------------+------+-----------+----+-----------+------------+-----+------------+----+----------+-------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|SALES|ORDERDATE|DAYS_SINCE_LASTORDER|STATUS|PRODUCTLINE|MSRP|PRODUCTCODE|CUSTOMERNAME|PHONE|ADDRESSLINE1|CITY|POSTALCODE|COUNTRY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-----+---------+--------------------+------+-----------+----+-----------+------------+-----+------------+----+----------+-------+---------------+----------------+--------+
+-----------+---------------+---------+---------------+-----+---------+--------------------+------+-----------+----+-----------+------------+-----+------------+----+----------+-------+---------------+----------------+--------+



In [36]:
# e. Write a Spark SQL query to find the latest order date for each customer.
df.groupBy('CUSTOMERNAME').agg(f.max('ORDERDATE').alias('MaxOrderDate')).sort(f.desc('MaxOrderDate')).show()

+--------------------+------------+
|        CUSTOMERNAME|MaxOrderDate|
+--------------------+------------+
|Euro Shopping Cha...|  31-05-2020|
|   La Rochelle Gifts|  31-05-2020|
|Oulu Toy Supplies...|  31-01-2020|
|Toys of Finland, Co.|  30-09-2019|
|     Vida Sport, Ltd|  30-08-2019|
|   Gifts4AllAges.com|  30-06-2019|
|Diecast Classics ...|  30-05-2020|
|          Petit Auto|  30-05-2020|
|  Reims Collectables|  30-03-2020|
|Australian Collec...|  29-11-2019|
|  Signal Gift Stores|  29-11-2019|
|Mini Gifts Distri...|  29-05-2020|
|Souveniers And Th...|  29-05-2020|
|Tekni Collectable...|  29-03-2019|
|West Coast Collec...|  29-01-2019|
|  Baane Mini Imports|  29-01-2018|
|Technics Stores Inc.|  28-10-2018|
|La Corne D'abonda...|  28-08-2019|
|     Gift Depot Inc.|  28-06-2019|
|Corrida Auto Repl...|  28-05-2018|
+--------------------+------------+
only showing top 20 rows

